# Table Schemas

In this example, I demonstrate features of the table creation interface.

In [1]:
import sys
sys.path.append('../')
import doctable
import pprint

The examples below demonstrate the use of doctable to create two tables, one for a list of people and one for their favorite colors. They include several features for creating database schemas, and I added comments in some places where the purpose may be unclear.

In [2]:
import datetime

@doctable.table_schema(
    table_name='color',
    constraints = [
        doctable.UniqueConstraint('name'),
    ]
)
class Color:
    name: str
    id: int = doctable.Column(
        column_args=doctable.ColumnArgs(
            primary_key=True,
            autoincrement=True,
        )
    )

# lets say we use this instead of an int
class PersonID(int):
    pass

# add table-level parameters to this decorator
@doctable.table_schema(
    table_name='person',
    indices = {
        'ind_name_birthday': doctable.Index('name', 'birthday', unique=True),
    },
    constraints = [ # these constraints are set on the database
        doctable.CheckConstraint('length(address) > 0'), # cannot have a blank address
        doctable.UniqueConstraint('birthday', 'fav_color'),
        doctable.ForeignKey(['fav_color'], ['color.name'], onupdate='CASCADE', ondelete='CASCADE'),
    ],
    frozen = True, # parameter passed to dataclasses.dataclass
)
class Person:
    name: str
    
    # default value will be "not provided" - good standardization
    address: str = doctable.Column(
        column_args=doctable.ColumnArgs(
            server_default='not provided',
        )
    )
    
    # provided as datetime, set to be indexed
    birthday: datetime.datetime = doctable.Column(
        column_args=doctable.ColumnArgs(
            index = True,
        )
    )
    
    # note that this has a foreign key constraint above
    fav_color: str = doctable.Column(
        column_args=doctable.ColumnArgs(
            nullable=False,
        )
    )
    
    id: PersonID = doctable.Column( # standard id column
        column_args=doctable.ColumnArgs(
            order=0, # will be the first column
            primary_key=True,
            autoincrement=True
        ),
    )
    
    # doctable will define default and onupdate when inserting into database
    added: datetime.datetime = doctable.Column(
        column_args=doctable.ColumnArgs(
            index=True,
            default=datetime.datetime.utcnow, 
            onupdate=datetime.datetime.utcnow
        )
    )
    
    # this property will not be stored in the database 
    #   - it acts like any other property
    @property
    def age(self):
        return datetime.datetime.now() - self.birthday
    
    
core = doctable.ConnectCore.open(
    target=':memory:', 
    dialect='sqlite'
)
# NOTE: weird error when trying to run this twice after defining containers
with core.begin_ddl() as emitter:
    core.enable_foreign_keys() # NOTE: NEEDED TO ENABLE FOREIGN KEYS
    color_tab = emitter.create_table_if_not_exists(container_type=Color)
    person_tab = emitter.create_table_if_not_exists(container_type=Person)
for col_info in person_tab.inspect_columns():
    print(f'{col_info["name"]}: {col_info["type"]}')

id: INTEGER
added: DATETIME
address: VARCHAR
birthday: DATETIME
fav_color: VARCHAR
name: VARCHAR


Insertion into the color table is fairly straightforward.

In [3]:
color_names = ['red', 'green', 'blue']
colors = [Color(name=name) for name in color_names]
with color_tab.query() as q:
    q.insert_multi(colors)
    for c in q.select():
        print(c)
    #print(q.select())

Color(name='red', id=1)
Color(name='green', id=2)
Color(name='blue', id=3)


Insertion into the person table is similar, and note that we see an exception if we try to insert a person with a favorite color that is not in the color table.

In [4]:
persons = [
    Person(name='John', birthday=datetime.datetime(1990, 1, 1), fav_color='red'),
    Person(name='Sue', birthday=datetime.datetime(1991, 1, 1), fav_color='green'),
    Person(name='Ren', birthday=datetime.datetime(1995, 1, 1), fav_color='blue'),
]
other_person = Person(
    name='Bob', 
    address='123 Main St', 
    birthday=datetime.datetime(1990, 1, 1), 
    fav_color='other', # NOTE: THIS WILL CAUSE AN ERROR (NOT IN COLOR TABLE)
)

import sqlalchemy.exc

sec_in_one_year = 24*60*60*365
with person_tab.query() as q:
    q.insert_multi(persons, ifnotunique='replace')
    
    try:
        q.insert_single(other_person, ifnotunique='replace')
        print(f'THIS SHOULD NOT APPEAR')
    except sqlalchemy.exc.IntegrityError as e:
        print(f'successfully threw exception: {e}')
    
    for p in q.select():
        print(f'{p.name} ({p.fav_color}): {p.age.total_seconds()//sec_in_one_year:0.0f} y/o')

successfully threw exception: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT OR REPLACE INTO person (added, address, birthday, fav_color, name) VALUES (?, ?, ?, ?, ?)]
[parameters: ('2023-11-11 19:13:08.275137', '123 Main St', '1990-01-01 00:00:00.000000', 'other', 'Bob')]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
John (red): 33 y/o
Sue (green): 32 y/o
Ren (blue): 28 y/o


The foreign key works as expected because we set `onupdate`: changing that value in the parent table will update the value in the child table.

In [5]:
with color_tab.query() as q:
    q.update_single(dict(name='reddish'), where=color_tab['name']=='red')
    for c in q.select():
        print(c)
        
with person_tab.query() as q:
    for p in q.select():
        print(f'{p.name} ({p.fav_color}): {p.age.total_seconds()//sec_in_one_year:0.0f} y/o')

Color(name='reddish', id=1)
Color(name='green', id=2)
Color(name='blue', id=3)
John (reddish): 33 y/o
Sue (green): 32 y/o
Ren (blue): 28 y/o
